In [1]:
import os, numpy as np
import pandas as pd

from sklearn import datasets

from sklearn.model_selection import train_test_split

import torch
from torch import nn
import torch.nn.functional as F

import skorch
from skorch import NeuralNetRegressor

torch.manual_seed(1960)

torch.set_default_tensor_type('torch.DoubleTensor')


# Build a PyTorch Model using Skorch

In [2]:
# for the moment, only experimenting with pytorch sequential models, 
# This is a limitation and we will try to see what can be made to make
# recurrent layers and convolutions usable inside sequential models.
# Functional models (with custom forward methods) (are not/will not be) supported anyway.


# This is a toy regression model with one hidden layer.
def create_model():
    hidden_units = 3
    model = nn.Sequential(
        nn.Linear(13, hidden_units),
        nn.Linear(hidden_units , 1))
    return model


In [3]:
boston  = datasets.load_boston()
train_X, test_X, train_y, test_y = train_test_split(boston.data, boston.target, train_size=0.8, test_size=0.2, random_state=1960)
train_y = train_y.reshape(-1, 1)
test_y = test_y.reshape(-1, 1)

In [4]:

net = skorch.NeuralNetRegressor(
    create_model(),
    optimizer=torch.optim.Adam,
    max_epochs=10,
)


print(train_X.shape , train_y.shape)
net.fit(train_X, train_y)

(404, 13) (404, 1)
  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1      297.5652      148.7383  0.0603
      2      121.4895       88.5588  0.0089
      3       86.3258       87.3114  0.0083
      4      101.9769      107.4249  0.0084
      5      105.2715       85.5098  0.0081
      6       88.7895       73.8404  0.0081
      7       72.6823       68.5698  0.0080
      8       67.9871       68.8389  0.0082
      9       70.4357       74.6676  0.0078
     10       71.1231       69.4323  0.0088


<class 'skorch.regressor.NeuralNetRegressor'>[initialized](
  module_=Sequential(
    (0): Linear(in_features=13, out_features=3, bias=True)
    (1): Linear(in_features=3, out_features=1, bias=True)
  ),
)

In [5]:
print(test_X.shape)
preds = net.predict(test_X[0,:].reshape(1,13))
print(preds)


(102, 13)
[[20.00808598]]


# Generate SQL Code from the Model

In [6]:
import json, requests, base64, pickle, sys

sys.setrecursionlimit(200000)

# Pickle the model and send it to the SQL generation web service
# Get back the  SQL code.
def test_ws_sql_gen(pickle_data):
    WS_URL="http://localhost:1888/model"
    b64_data = base64.b64encode(pickle_data).decode('utf-8')
    data={"Name":"model1", "PickleData":b64_data , "SQLDialect":"postgresql"}
    r = requests.post(WS_URL, json=data)
    # print(r.__dict__)
    content = r.json()
    # print(content)
    lSQL = content["model"]["SQLGenrationResult"][0]["SQL"]
    return lSQL;



In [7]:
pickle_data = pickle.dumps(net)
lSQL = test_ws_sql_gen(pickle_data)
# print(lSQL[0:2000])

In [8]:
print(lSQL)

WITH pytorch_input AS 
(SELECT "ADS"."KEY" AS "KEY", "ADS"."Feature_0" AS "Feature_0", "ADS"."Feature_1" AS "Feature_1", "ADS"."Feature_2" AS "Feature_2", "ADS"."Feature_3" AS "Feature_3", "ADS"."Feature_4" AS "Feature_4", "ADS"."Feature_5" AS "Feature_5", "ADS"."Feature_6" AS "Feature_6", "ADS"."Feature_7" AS "Feature_7", "ADS"."Feature_8" AS "Feature_8", "ADS"."Feature_9" AS "Feature_9", "ADS"."Feature_10" AS "Feature_10", "ADS"."Feature_11" AS "Feature_11", "ADS"."Feature_12" AS "Feature_12" 
FROM "INPUT_DATA" AS "ADS"), 
pytorch_input_1 AS 
(SELECT pytorch_input."KEY" AS "KEY", pytorch_input."Feature_0" AS "Feature_0", pytorch_input."Feature_1" AS "Feature_1", pytorch_input."Feature_2" AS "Feature_2", pytorch_input."Feature_3" AS "Feature_3", pytorch_input."Feature_4" AS "Feature_4", pytorch_input."Feature_5" AS "Feature_5", pytorch_input."Feature_6" AS "Feature_6", pytorch_input."Feature_7" AS "Feature_7", pytorch_input."Feature_8" AS "Feature_8", pytorch_input."Feature_9" AS "Fea

# Execute the SQL Code

In [9]:
# save the dataset in a database table


import sqlalchemy as sa

#engine = sa.create_engine('sqlite://' , echo=False)
engine = sa.create_engine("postgresql://db:db@localhost/db?port=5432", echo=False)
conn = engine.connect()

lTable = pd.DataFrame(boston.data);
NC = boston.data.shape[1]
lFeatures = ['Feature_' + str(x) for x in range(NC)]
lTable.columns = lFeatures
lTable['TGT'] = boston.target
lTable['KEY'] = range(boston.data.shape[0])
lTable.to_sql("INPUT_DATA" , conn,   if_exists='replace', index=False)

In [10]:
sql_output = pd.read_sql(lSQL , conn);
conn.close()

In [11]:
sql_output.sample(12, random_state=1960)

,KEY,Estimator
230,230,20.008086
112,112,22.846729
125,125,16.048917
9,9,20.526287
213,213,20.618331
109,109,22.388635
127,127,22.155289
244,244,22.027804
406,406,19.462396
490,490,23.692042


# PyTorch Prediction

In [12]:
pytorch_output = pd.DataFrame()
pytorch_output_key = pd.DataFrame(list(range(boston.data.shape[0])), columns=['KEY']);
pytorch_output_estimator = pd.DataFrame(net.predict(boston.data), columns=['Estimator'])
pytorch_output = pytorch_output_key
pytorch_output['Estimator'] = pytorch_output_estimator
pytorch_output.sample(12, random_state=1960)

,KEY,Estimator
230,230,20.008086
112,112,22.846729
125,125,16.048917
9,9,20.526287
213,213,20.618331
109,109,22.388635
127,127,22.155289
244,244,22.027804
406,406,19.462396
490,490,23.692042


# Comparing the SQL and PyTorch Predictions

In [13]:
sql_pytorch_join = pytorch_output.join(sql_output , how='left', on='KEY', lsuffix='_pytorch', rsuffix='_sql')
sql_pytorch_join['Error'] = sql_pytorch_join.Estimator_sql - sql_pytorch_join.Estimator_pytorch

In [14]:
sql_pytorch_join.head(12)

,KEY_pytorch,Estimator_pytorch,KEY_sql,Estimator_sql,Error
0,0,24.022744,0,24.022744,3.552714e-15
1,1,20.513554,1,20.513554,2.486900e-14
2,2,21.277775,2,21.277775,4.263256e-14
3,3,21.556214,3,21.556214,7.105427e-15
4,4,21.273130,4,21.273130,1.776357e-14
5,5,21.067930,5,21.067930,7.105427e-15
6,6,21.596923,6,21.596923,4.618528e-14
7,7,20.555443,7,20.555443,-1.065814e-14
8,8,18.677997,8,18.677997,-2.486900e-14
9,9,20.526287,9,20.526287,-3.552714e-15


In [15]:
sql_pytorch_join.describe()

,KEY_pytorch,Estimator_pytorch,KEY_sql,Estimator_sql,Error
count,506.000000,506.000000,506.000000,506.000000,5.060000e+02
mean,252.500000,21.191186,252.500000,21.191186,-1.598853e-15
std,146.213884,4.392024,146.213884,4.392024,2.799861e-14
min,0.000000,-0.190742,0.000000,-0.190742,-5.684342e-14
25%,126.250000,20.155486,126.250000,20.155486,-2.486900e-14
50%,252.500000,21.593711,252.500000,21.593711,-8.881784e-16
75%,378.750000,23.391669,378.750000,23.391669,2.131628e-14
max,505.000000,31.899223,505.000000,31.899223,5.329071e-14


In [16]:
sql_pytorch_join.Error.describe()

count    5.060000e+02
mean    -1.598853e-15
std      2.799861e-14
min     -5.684342e-14
25%     -2.486900e-14
50%     -8.881784e-16
75%      2.131628e-14
max      5.329071e-14
Name: Error, dtype: float64